In [ ]:
import os

# Import packages needed to extract road network using a polygon
import osmnx as ox
import geopandas


# Import packages to extract buildings
import matplotlib.pyplot as plt
import osmnx as ox
from IPython.display import Image

# Other packages
import networkx as nx # to work with graphs
import pandas as pd
import numpy as np
import scipy as sp # for the adjacency matrix

%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

In [ ]:
# Read polygons
FL11 = geopandas.read_file("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Polygons\\FL11_geo.shp")
# Extract just the geometry (shapely object) part
FL11_geo = FL11['geometry'].iloc[0]
# Clean it with a buffer
FL11_geo = FL11_geo.buffer(0)
FL11_geo.is_valid

In [ ]:
# Pull the roads with OSMnx
G = ox.graph_from_polygon(FL11_geo)

# Plot the network and print basic statistics
ox.plot_graph(G)
ox.stats.basic_stats(G, area=None, clean_intersects=False, tolerance=15, circuity_dist='gc')


In [ ]:
osmids = list(G.nodes)
G = nx.relabel.convert_node_labels_to_integers(G)
osmid_values = {k:v for k, v in zip(G.nodes, osmids)}
nx.set_node_attributes(G, osmid_values, 'osmid')

In [ ]:
# See info in nodes
G.nodes(data=True)

In [ ]:
# Read shp
Elevation = geopandas.read_file("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Elevation\\FL11_nodes.shp")
type(Elevation)

In [ ]:
Elevation =  pd.DataFrame(Elevation)
type(Elevation)

In [ ]:
E = Elevation.elevations
len(E)

In [ ]:
# Add to each node its respective elevation
for node in G.nodes():
    G.add_node(node,elevation=E[node])
    

In [ ]:
G.nodes(data=True)

In [ ]:
# Save street network as graph
ox.save_graphml(G, filepath='C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Example\\FL11_network.graphml')


In [ ]:
G2 = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
type(G2)

In [ ]:
# Stair histogram for elevations
fig = plt.hist(G2.elevation, bins = 20, histtype='step')
plt.title('FL11')
plt.xlabel("Elevation")
plt.ylabel("Frequency")
plt.savefig("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Example\\FL11.png",facecolor='w')

In [ ]:
# Create adjacency matrix 

A = nx.adjacency_matrix(G)

print(A.todense())